In [1]:
import numpy as np
from numpy.polynomial import polynomial as poly

def polymul(x, y, modulus, poly_mod):
    # Multiply two polynoms
    return np.int64(
        np.round(poly.polydiv(poly.polymul(x, y) % modulus, poly_mod)[1] % modulus)
    )

def polyadd(x, y, modulus, poly_mod):
    # Add two polynoms
    return np.int64(
        np.round(poly.polydiv(poly.polyadd(x, y) % modulus, poly_mod)[1] % modulus)
    )

def gen_binary_poly(size):
    # Generates a polynomial with coeffecients 0 or 1
    # size is the number of coeffcients
    # array of coefficients with the coefficient[i] being the coefficient of x ^ i
    return np.random.randint(0, 2, size, dtype=np.int64)


def gen_uniform_poly(size, modulus):
    # Generates a polynomial with coeffecients being integers in Z_modulus
    return np.random.randint(0, modulus, size, dtype=np.int64)

def gen_normal_poly(size):
    # Generates a polynomial with coeffecients in a normal distribution of mean 0 and a standard deviation of 2, then discretize it.
    return np.int64(np.random.normal(0, 2, size=size))

def keygen(size, modulus, poly_mod):
    # Generate a public and secret keys
    sk = gen_binary_poly(size)
    a = gen_uniform_poly(size, modulus)
    e = gen_normal_poly(size)
    b = polyadd(polymul(-a, sk, modulus, poly_mod), -e, modulus, poly_mod)
    return (b, a), sk

In [2]:
def encrypt(pk, size, q, t, poly_mod, pt):
#     pk: public-key.
#     size: size of polynomials.
#     q: ciphertext modulus.
#     t: plaintext modulus.
#     poly_mod: polynomial modulus.
#     pt: integer to be encrypted.

    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    e1 = gen_normal_poly(size)
    e2 = gen_normal_poly(size)
    u = gen_binary_poly(size)
    ct0 = polyadd(
            polyadd(
                polymul(pk[0], u, q, poly_mod),
                e1, q, poly_mod),
            scaled_m, q, poly_mod
        )
    ct1 = polyadd(
            polymul(pk[1], u, q, poly_mod),
            e2, q, poly_mod
        )
    return (ct0, ct1)

In [3]:
def decrypt(sk, size, q, t, poly_mod, ct):
#     sk: secret-key.
#     size: size of polynomials.
#     q: ciphertext modulus.
#     t: plaintext modulus.
#     poly_mod: polynomial modulus.
#     ct: ciphertext.

    scaled_pt = polyadd(
            polymul(ct[1], sk, q, poly_mod),
            ct[0], q, poly_mod
        )
    decrypted_poly = np.round(scaled_pt * t / q) % t
    return int(decrypted_poly[0])

In [4]:
def add_plain(ct, pt, q, t, poly_mod):
    # Add a ciphertext and a plaintext.
    
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    delta = q // t
    scaled_m = delta * m  % q
    new_ct0 = polyadd(ct[0], scaled_m, q, poly_mod)
    return (new_ct0, ct[1])

def mul_plain(ct, pt, q, t, poly_mod):
    # Multiply a ciphertext and a plaintext.
    size = len(poly_mod) - 1
    # encode the integer into a plaintext polynomial
    m = np.array([pt] + [0] * (size - 1), dtype=np.int64) % t
    new_c0 = polymul(ct[0], m, q, poly_mod)
    new_c1 = polymul(ct[1], m, q, poly_mod)
    return (new_c0, new_c1)

In [5]:
# polynomial modulus degree
n = 2**4
# ciphertext modulus
q = 2**15
# plaintext modulus
t = 2**8
# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])
# Key generation
pk, sk = keygen(n, q, poly_mod)
# Encryption
pt1, pt2 = 73, 20
cst1, cst2 = 7, 5
ct1 = encrypt(pk, n, q, t, poly_mod, pt1)
ct2 = encrypt(pk, n, q, t, poly_mod, pt2)

print("[+] Ciphertext ct1({}):".format(pt1))
print("")
print("\t ct1_0:", ct1[0])
print("\t ct1_1:", ct1[1])
print("")
print("[+] Ciphertext ct2({}):".format(pt2))
print("")
print("\t ct1_0:", ct2[0])
print("\t ct1_1:", ct2[1])
print("")

# Evaluation
ct3 = add_plain(ct1, cst1, q, t, poly_mod)
ct4 = mul_plain(ct2, cst2, q, t, poly_mod)

# Decryption
decrypted_ct3 = decrypt(sk, n, q, t, poly_mod, ct3)
decrypted_ct4 = decrypt(sk, n, q, t, poly_mod, ct4)

print("[+] Decrypted ct3(ct1 + {}): {}".format(cst1, decrypted_ct3))
print("[+] Decrypted ct4(ct2 * {}): {}".format(cst2, decrypted_ct4))

[+] Ciphertext ct1(73):

	 ct1_0: [17993 22281  3738  3065 13263  9532  8980  1113 11442 20632  5566 28329
 18531 15376 24511 24126]
	 ct1_1: [18608 13376 24020  1678 29249 17496   828 29146 25057 25842  9884 12511
 17590  7345  5625 27941]

[+] Ciphertext ct2(20):

	 ct1_0: [11155 31091 24828 13271 20658 10422  6476 15745  9407 30896 18459 20019
 23446 15371 26851 10842]
	 ct1_1: [ 9665 23083  5039 18210 28380 23279 13588 12760 25331 10141 21910 15506
 13989 23012 29866  3692]

[+] Decrypted ct3(ct1 + 7): 80
[+] Decrypted ct4(ct2 * 5): 100
